In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [3]:
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp').astype(np.int8)

available_labels_sided = [labels_sided[i-1] for i in np.unique(atlas_volume) if i > 0]
available_labels_unsided = set([labelMap_sidedToUnsided[name] for name in available_labels_sided ])

In [4]:
available_labels_sided = ['RtTg',
 'VLL_L',
 'VLL_R',
 'Tz_L',
 'Tz_R',
 'LC_L',
 'LC_R',
 '7n_L',
 '7n_R',
 'Amb_L',
 'Amb_R',
 '12N',
 'AP',
 '6N_L',
 '6N_R',
 '5N_L',
 '5N_R',
 '7N_L',
 '7N_R',
 'R_L',
 'R_R',
 'Pn_L',
 'Pn_R',
 'LRt_L',
 'LRt_R']

In [ ]:
# Convert projected volumes to meshes

In [ ]:
def fill_sparse_volume(volume_sparse):
    '''
    Sometimes, volumes formed by projecting from other volumes result in holes in corresponding regions.
    This function fills the holes.
    '''

    volume = np.zeros_like(volume_sparse, np.int8)

    for z in range(volume_sparse.shape[-1]):
        for ind, cnts in find_contour_points(volume_sparse[..., z]).iteritems():
            cnt = cnts[np.argsort(map(len, cnts))[-1]]
            pts = points_inside_contour(cnt)
            volume[pts[:,1], pts[:,0], z] = ind
    return volume

In [ ]:
for stack in ['MD589', 'MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD603']:
    mesh_dir = create_if_not_exists(os.path.join(mesh_rootdir, stack))
    
    #################
    
    volume_sparse = bp.unpack_ndarray_file(volume_dir + '%(stack)s/%(stack)s_atlasProjectedVolume.bp' % {'stack':stack}).astype(np.int8)
    atlasProjected_volume = fill_sparse_volume(volume_sparse)
    del volume_sparse

    atlasProjectedVolume_landmark_bboxes = {name: bbox_3d(atlasProjected_volume == labels_sided_indices[name])
                               for name in available_labels_sided}
    
    ###################
    
    for name_s in available_labels_sided:

        print name_s

        xmin, xmax, ymin, ymax, zmin, zmax = atlasProjectedVolume_landmark_bboxes[name_s]

        vol = (atlasProjected_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)

        t = time.time()

        polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

        sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

        save_mesh_stl(polydata, mesh_dir + "/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})
        
    ####################
        
    volume_sparse = bp.unpack_ndarray_file(volume_dir + '%(stack)s/%(stack)s_localAdjustedVolume.bp' % {'stack':stack}).astype(np.int8)
    localAdjusted_volume = fill_sparse_volume(volume_sparse)
    del volume_sparse
    
    localAdjustedVolume_landmark_bboxes = {name: bbox_3d(localAdjusted_volume == labels_sided_indices[name])
                               for name in available_labels_sided}

    ######################
    
    for name_s in available_labels_sided:

        print name_s

        xmin, xmax, ymin, ymax, zmin, zmax = localAdjustedVolume_landmark_bboxes[name_s]

        vol = (localAdjusted_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)

        t = time.time()

        polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

        sys.stderr.write('time: %.2f\n' % (time.time() - t) )

        save_mesh_stl(polydata, mesh_dir + "/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})

In [5]:
stack = 'MD594'

In [6]:
# Load atlasProjected volume

atlasProjected_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    atlasProjected_polydata_list[name_s] = polydata

In [7]:
# Load localAdjusted volume

localAdjusted_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    localAdjusted_polydata_list[name_s] = polydata

In [7]:
# Load thumbnail volume

tb_volume = bp.unpack_ndarray_file(volume_dir + "/%(stack)s/%(stack)s_thumbnailVolume.bp" % {'stack': stack})

xmin, xmax, ymin, ymax, zmin, zmax = np.loadtxt(volume_dir + "/%(stack)s/%(stack)s_scoreVolume_limits.txt" % {'stack': stack}, np.int)

from scipy.ndimage.interpolation import zoom
tb_volume_scaledToScoreVolume = img_as_ubyte(zoom(tb_volume, 2)[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1])
bp.pack_ndarray_file(tb_volume_scaledToScoreVolume, 
                     volume_dir + "/%(stack)s/%(stack)s_thumbnailVolume_scaledToScoreVolume.bp" % {'stack': stack})

KeyboardInterrupt: 

In [11]:
# Load contour

from registration_utilities import *

# load in annotation space
vertices, faces = load_mesh_stl(mesh_rootdir + "/%(stack)s/%(stack)s_%(name)s_gaussianSmoothed.stl" % \
                                {'stack': stack, 'name': 'outerContour'})

ann_xmin, ann_xmax, ann_ymin, ann_ymax, ann_zmin, ann_zmax = \
np.loadtxt(volume_dir + '%(stack)s/volume_%(stack)s_annotation_withOuterContour_limits.txt' % {'stack': stack}, dtype=np.int)

sco_xmin, sco_xmax, sco_ymin, sco_ymax, sco_zmin, sco_zmax = \
np.loadtxt(volume_dir + '%(stack)s/%(stack)s_scoreVolume_limits.txt' % {'stack': stack}, dtype=np.int)

# convert to score volume space
# vertices_alignedToScoreVolume = vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin) - (100,50,-50)
vertices_alignedToScoreVolume = vertices + (ann_xmin,ann_ymin,ann_zmin) - (sco_xmin,sco_ymin,sco_zmin)

polydata_outerContour = mesh_to_polydata(vertices_alignedToScoreVolume, faces)

contour_mapper = vtk.vtkPolyDataMapper()
contour_mapper.SetInputData(polydata_outerContour)

contour_actor = vtk.vtkActor()
contour_actor.SetMapper(contour_mapper)
# contour_actor.GetProperty().SetRepresentationToWireframe()
contour_actor.GetProperty().SetColor(1,1,1)
contour_actor.GetProperty().SetOpacity(.2)

fill point array: 0.02 seconds
fill cell array: 0.00 seconds


In [19]:
# Load thumbnail, only show several sections

tb_volume = load_thumbnail_volume(stack, convert_to_scoreSpace=True)
tb_volume2 = np.zeros_like(tb_volume)
for z in range(230, 500, 80):
    tb_volume2[:, :, z:z+5] = tb_volume[:,:,z:z+5]

In [20]:
mesh_actors = [actor_mesh(polydata, colors[labelMap_sidedToUnsided[name_s]], opacity=1.) 
               for name_s, polydata in localAdjusted_polydata_list.iteritems()]
# smaller opacity causes erroneous rendering

launch_vtk(mesh_actors + [actor_volume(tb_volume2, 'tb', origin=(0,-200,0)), contour_actor])

In [ ]:
name_s = '7N_L'

mesh_actor = actor_mesh(localAdjusted_polydata_list[name_s], colors[labelMap_sidedToUnsided[name_s]], opacity=.7)

score_volume_actor = actor_volume(load_score_volume(stack, labelMap_sidedToUnsided[name_s]), 'score')

launch_vtk([mesh_actor, score_volume_actor])

In [ ]:
oneStructure_screenshot_dir = create_if_not_exists(volume_dir + '/%(stack)s/atlasProjected_score_screenshots' % {'stack':stack})

for name_s in available_labels_sided:

    mesh_actor = actor_mesh(atlasProjected_polydata_list[name_s], colors[labelMap_sidedToUnsided[name_s]], opacity=.7)

    score_volume_actor = actor_volume(load_score_volume(stack, labelMap_sidedToUnsided[name_s]), 'score')

    launch_vtk([mesh_actor, score_volume_actor], interactive=False, 
               snapshot_fn=oneStructure_screenshot_dir + '/atlasProjected_score_%(name_s)s.png' % \
               {'stack': stack, 'name_s': name_s})

In [ ]:
oneStructure_screenshot_dir = create_if_not_exists(volume_dir + '/%(stack)s/localAdjusted_score_screenshots' % {'stack':stack})

for name_s in available_labels_sided:

    mesh_actor = actor_mesh(localAdjusted_polydata_list[name_s], colors[labelMap_sidedToUnsided[name_s]], opacity=.7)

    score_volume_actor = actor_volume(load_score_volume(stack, labelMap_sidedToUnsided[name_s]), 'score')

    launch_vtk([mesh_actor, score_volume_actor], interactive=False, 
               snapshot_fn=oneStructure_screenshot_dir + '/localAdjusted_score_%(name_s)s.png' % \
               {'stack': stack, 'name_s': name_s})